## Algoritmos en PYSPARK BOW.hs
##### Alumna:Milagros Yarahuaman Rojas
#### Cod:171071

#### - BAGOFWORDS

In [103]:
def bagofwords(Doc):
    """
        -Normaliza a minusculas,
        -Filtra letras a mayores a 2,
        -Remueve espacios en blanco
        Args:
            texto (str): A string.
        Returns:
            [[str]]: Arreglo que contiene documentos que son [str]
    """
    Temp=Doc.strip().lower().split()
    return list(filter(lambda x:len(x)>2,Temp))

In [105]:
Corpus=["messi messi messi ronaldo ronaldo balon","messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
Texto.take(2)

[['messi', 'messi', 'messi', 'ronaldo', 'ronaldo', 'balon'],
 ['messi', 'ronaldo', 'futbol', 'futbol', 'futbol']]

#### - TF

In [5]:
import math
def TF(Doc):
    """
        Funcion que calcula el tf de un documento 
        Args:
            Doc (str): un Str que es un documento que contiene palabras

        Returns:
            (str,double): un RDD contiene (palabra,double) tupla.
    """
    #Obtenemos el total de palabras de doc
    total=len(Doc)
    #Lista sin repeticiones
    DocSR=list(set(Doc))
    #Obtenemos el tf para cada palabra
    mapeo=list(map(lambda x:(x,Doc.count(x)),DocSR))
    tf=list(map(lambda x:(x[0],1+math.log10(x[1]/total)),mapeo))
    return (tf)

In [106]:
Corpus=["messi messi messi ronaldo ronaldo balon","messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoTF.take(2)

[[('ronaldo', 0.5228787452803375),
  ('messi', 0.6989700043360187),
  ('balon', 0.22184874961635637)],
 [('ronaldo', 0.30102999566398125),
  ('futbol', 0.7781512503836436),
  ('messi', 0.30102999566398125)]]

#### - IDF

In [6]:
import math
def IDF(Doc,corpus):
    """
        Funcion que calcula el IDF de un corpus de TF 

        Args:
            Doc [[(str,double)]]: texto TF
            Doc [str]:texto corpus basico procesado con bagofwords

        Returns:
            (str,double): corpus con IDF de cada palabra sin repeticiones
    """
    DocP=Doc.flatMap(lambda x:x)
    DocP=DocP.map(lambda x:x[0]).distinct()
    #Total documentos en corpus
    total=corpus.count()
    corpus2=corpus.collect()
    #Extraer cantida de veces de cada palabra en los documentos
    IDF=DocP.map(lambda x:list(map(lambda y:(x,1 if x in y else 0),corpus2)))
    IDF2=IDF.flatMap(lambda x:x)
    IDF2=IDF2.groupByKey().map(lambda x:(x[0],sum(x[1])))
    IDF2=IDF2.map(lambda x:(x[0],math.log10(1+total/x[1])))
    return IDF2

In [107]:
Corpus=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoIDF=IDF(TextoTF,Texto)
TextoIDF.collect()

[('ronaldo', 0.3010299956639812),
 ('messi', 0.3010299956639812),
 ('balon', 0.47712125471966244),
 ('futbol', 0.47712125471966244)]

#### - TFIDF

In [108]:
def RecIDF(word,corpusIDF):
    """
        Funcion que recuperar el valor IDF de una palabra
        Args:
            word str:palabra a buscar
            Doc [(str,double)]:texto IDF
        Returns:
            double: IDF numerico de una palbara
    """
    L=corpusIDF
    EL=list(filter(lambda x:x[0]==word,L))
    return EL[0][1]

In [48]:
import math

def TFIDF(TextoTF,TextoIDF):
    """
        Funcion que calcula el IDF para cada Doc de TF
        Args:
            Doc [[(str,double)]]: texto TF
            Doc [(str,double)]:texto IDF

        Returns:
            [(str,double)]: texto con el TFIDF de cada palabra en DOC
    """
    corpusIDF=TextoIDF.collect()
    TFIDF=TextoTF.map(lambda x:list(map((lambda y:(y[0],y[1]*RecIDF(y[0],corpusIDF))),x)))
    return  TFIDF

In [109]:
Corpus=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoIDF=IDF(TextoTF,Texto)
TextoTFIDF=TFIDF(TextoTF,TextoIDF)
TextoTFIDF.collect()

[[('ronaldo', 0.15740218642452794),
  ('messi', 0.21041093737452465),
  ('balon', 0.10584875377494418)],
 [('ronaldo', 0.09061905828945656),
  ('futbol', 0.37127250094471825),
  ('messi', 0.09061905828945656)]]

#### - nGrams

In [110]:
def taken(n,tokens):
    """
        Funcion para recuperar n elementos de un array no sc
        Args:
            n int:cantidad a recuperar
            [tokens]:arreglo
        Returs:
            Str con n elementos del arreglo
    """
    if len(tokens)>=n:
        tokens=list(map(lambda x:str(x),tokens))
        val=((" ").join(tokens[0:n]))
    else:
        val=""
    return val
def tails(tokens):
    """
        Funcion para generar una lista con todas las colas posibles
        Args:
            [tokens]:arreglo
        Returs:
            [tokens]:arreglo con un elemento menos
    """
    tailsss=[tokens]
    for i in range(len(tokens)-1):
        tailsss.append(tailsss[-1][1:])
    return tailsss
def nGrams(n,tokens):
    """
        Funcion que genera ngrams de una secuencia de tokens
        Args:
            n int: cantidad de gramas
            [str]:lista de tokens

        Returns:
            [str]: lista de tokens
    """
    tokens=tokens.split()
    tokens=tails(tokens)
    Tok=list(map(lambda x:taken(n,x),tokens))
    #elimino los vacios
    Tok=list(filter(lambda x:len(x)>0,Tok))
    return Tok

In [111]:
Corpus=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
CorpusP=sc.parallelize(Corpus,4)
#Calculamos Bigramas
Bigrama=CorpusP.map(lambda x:nGrams(2,x))
print('Bigrama',Bigrama.collect())
#Calculamos Trigramas
Trigramas=CorpusP.map(lambda x:nGrams(3,x))
print("Trigramas",Trigramas.collect())
#Calculamos Tetragramas
Tetragramas=CorpusP.map(lambda x:nGrams(4,x))
print("Tetragramas",Tetragramas.collect())

Bigrama [['messi messi', 'messi messi', 'messi ronaldo', 'ronaldo ronaldo', 'ronaldo balon'], ['messi ronaldo', 'ronaldo futbol', 'futbol futbol', 'futbol futbol']]
Trigramas [['messi messi messi', 'messi messi ronaldo', 'messi ronaldo ronaldo', 'ronaldo ronaldo balon'], ['messi ronaldo futbol', 'ronaldo futbol futbol', 'futbol futbol futbol']]
Tetragramas [['messi messi messi ronaldo', 'messi messi ronaldo ronaldo', 'messi ronaldo ronaldo balon'], ['messi ronaldo futbol futbol', 'ronaldo futbol futbol futbol']]
